# Single Atom Calculation

In [ ]:
# Uncomment the next line if you have ipympl installed and want interactive plots
# %matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize

import pairinteraction.$PI_DTYPE as pi
from pairinteraction.visualization.colormaps import alphamagma

## Parameters

In [ ]:
steps = $STEPS
Distance = np.linspace($DISTANCE_MIN, $DISTANCE_MAX, steps)
Angle = np.linspace($ANGLE_MIN, $ANGLE_MAX, steps)

## Atom 1

In [ ]:
ket1 = pi.KetAtom($SPECIES_0$QUANTUM_NUMBERS_0)
print(f"State of interest 1: {ket1}")

basis1 = pi.BasisAtom(ket1.species$QUANTUM_NUMBERS_RESTRICTIONS_0)
print(str(basis1))
print(f" ⇒ Basis 1 consists of {basis1.number_of_kets} kets")

system1 = pi.SystemAtom(basis1)
system1.set_electric_field([$EX_VALUE, $EY_VALUE, $EZ_VALUE], unit="V/cm")
system1.set_magnetic_field([$BX_VALUE, $BY_VALUE, $BZ_VALUE], unit="G")

## Atom 2

In [ ]:
ket2 = pi.KetAtom($SPECIES_1$QUANTUM_NUMBERS_1)
print(f"State of interest 2: {ket2}")

basis2 = pi.BasisAtom(ket2.species$QUANTUM_NUMBERS_RESTRICTIONS_1)
print(str(basis2))
print(f" ⇒ Basis 2 consists of {basis2.number_of_kets} kets")

system2 = pi.SystemAtom(basis2)
system2.set_electric_field([$EX_VALUE, $EY_VALUE, $EZ_VALUE], unit="V/cm")
system2.set_magnetic_field([$BX_VALUE, $BY_VALUE, $BZ_VALUE], unit="G")

## Pair Basis

In [ ]:
pi.diagonalize([system1, system2]$DIAGONALIZE_KWARGS)
pair_energy = sum(system.get_corresponding_energy(ket, "GHz") for ket, system in zip([ket1, ket2], [system1, system2]))

basis_pair = pi.BasisPair(
    [system1, system2],
    energy=(pair_energy - $PAIR_ENERGY_DELTA, pair_energy + $PAIR_ENERGY_DELTA),
    energy_unit="GHz",
)
print(str(basis_pair))
print(f" ⇒ Basis Pair consists of {basis_pair.number_of_kets} kets")

## Pair System

In [ ]:
system_pair_list = []

for i in range(steps):
    system_pair = pi.SystemPair(basis_pair)
    system_pair.set_distance(Distance[i], Angle[i], unit="micrometer")
    system_pair.set_interaction_order($MULTIPOLE_ORDER)
    system_pair_list.append(system_pair)

pi.diagonalize(system_pair_list$DIAGONALIZE_KWARGS$DIAGONALIZE_ENERGY_RANGE_KWARGS)

In [ ]:
energies = [system.get_eigenenergies(unit="GHz") - pair_energy for system in system_pair_list]
overlaps = [system.get_eigenbasis().get_overlaps([ket1, ket2]) for system in system_pair_list]

## Plot

In [ ]:
fig, ax = plt.subplots()

x_values = $X_VALUES
ax.set_xlabel($X_LABEL)
ax.set_ylabel("Energy [GHz]")

try:
    ax.plot(x_values, np.array(energies), c="0.9", lw=0.25, zorder=-10)
except ValueError:  # inhomogeneous shape -> no simple line plot possible
    for x, es in zip(x_values, energies):
        ax.plot([x] * len(es), es, c="0.9", ls="None", marker=".", zorder=-10)

min_overlap = 0.0001
for x, es, os in zip(x_values, energies, overlaps):
    inds = np.argwhere(os > min_overlap).flatten()
    inds = inds[np.argsort(os[inds])]
    if len(inds) > 0:
        ax.scatter([x] * len(es[inds]), es[inds], c=os[inds], s=15, vmin=0, vmax=1, cmap=alphamagma)

mappable = plt.cm.ScalarMappable(cmap=alphamagma, norm=Normalize(vmin=0, vmax=1))
fig.colorbar(mappable, ax=ax, label="Overlap with state of interest")

plt.show()